#### Import relevant modules

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from keras.utils import np_utils
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

#### Import Dataset

In [ ]:
# Class names for different classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

# Load training data, labels; and testing data and their true labels
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#Hot encode labels
train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)

# Normalize pixel values between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

#Just checking
print("Train images size:", train_images.shape)
print("Train labels size:", train_labels.shape)
print("Test images size:", test_images.shape)
print("Test label size:", test_labels.shape)

#All images are 32x32 pixels wide


#### Visualize dataset

In [ ]:
%matplotlib inline
#Show first 25 training images below
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    x = np.array(train_labels[i]).tolist()
    index = x.index(1.)
    plt.xlabel(class_names[index])

#### Download MobileNetV2, remove output layer

In [ ]:
MobileNetV2 = MobileNetV2(include_top=False,
                input_shape=(96,96,3),
                  weights="imagenet",
                  classes=10,
                   alpha=1.0,
                   input_tensor=None)

##### Create model

In [ ]:
Model = tensorflow.keras.Sequential()

#Since MobileNetV2 takes image of min 96x96, add Conv2DTranspose layer to upscale image
Model.add(layers.Conv2DTranspose(3,(1,1),strides=(3,3), input_shape=(32,32,3)))
#Next, add MobileNetV2 layer
Model.add(MobileNetV2)
Model.add(layers.Flatten())

#### Extend the MobileNetV2 model by adding at least one hidden layer (dense, convolution or any other type of layer). Also attach one final output layer

In [ ]:
Model.add(layers.Dense(128, activation = 'relu')) #First hidden layer
Model.add(layers.Dense(128, activation = 'relu')) #Second hidden layer
Model.add(layers.Dropout(0.2)) #Third hidden layer
Model.add(layers.Dense(128, activation = 'relu')) #Fourth hidden layer
Model.add(layers.Dense(128, activation = 'relu')) #Fifth hidden layer
Model.add(layers.Dense(10, activation=tensorflow.nn.softmax)) #Output layer



#### Create callback function to end training when overfitting occurs (validation accuracy starts dropping)

In [ ]:
#Once validation loss stops decreasing for three epochs in a row, end training
#This is to prevent overfiting
early_stop = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

#### Add the appropriate loss function, compile and train the modified network to classify the CIFAR-10 dataset

In [ ]:
Model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#Used categorical_crossentropy as the y labels are hot encoded

In [ ]:
history = Model.fit(train_images, 
          train_labels,
          epochs=30,
          batch_size=32,
          verbose=1,
          validation_data=(test_images,test_labels),
          callbacks = [early_stop]
       )

#### Plot the loss function value with respect to the epoch number on the training data

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss function value on training data')
plt.xlabel('epoch')
plt.show()

#### Plot the accuracy (on training data) with respect to the epoch number

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(acc,label='Training Accuracy')
plt.plot(val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Model Training and Validation Accuracy')
plt.xlabel('epoch')
plt.show()

#### Show the accuracy of the trained claissifer over the entire testing dataset

In [ ]:
test_loss, test_acc = Model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)